In [ ]:
import pandas as pd
import re

In [ ]:
!pip install sumy nltk konlpy

In [ ]:
news=pd.read_csv('/content/drive/MyDrive/2024공모전/KB_AI/df_news.csv')
news.head(10)

,corp_name,date,title,content
0,AJ네트웍스,2024-07-24 16:54:10,"[하이일드 조달 노트] '차입구조 개선' AJ네트웍스, '로봇사업 강화' 기반 마련",[\n이 기사는 2024년 07월 19일 15시 40분 넘버스에 발행된 기사입니다....
1,AJ네트웍스,2024-07-12 13:22:15,"한신평, AJ네트웍스 BBB+로 유지",[\n\t\t\t [파이낸셜뉴스] 한국신용평가는 AJ네트웍스의 신용등급을 BBB+...
2,AJ네트웍스,2024-07-17 18:10:07,"[마켓인]등급 하향 우려에도…풀무원, 신종자본증권 목표액 조달","[\n풀무원, 신종자본증권 700억 모집에 980억 주문“당분간 재무부담 완화 쉽지..."
3,AJ네트웍스,2024-07-04 10:33:10,"[이호 기자의 마켓ON]SK지오센트릭·대신에프앤아이·AJ네트웍스, 나란히 회사채 발행","[\n\t\t\tSK지오센트릭과 대신에프앤아이, AJ네트웍스가 나란히 회사채를 발행..."
4,AJ네트웍스,2024-06-28 09:51:18,"AJ네트웍스, 300억 규모 녹색채권 발행",[\n\n\n\n\nAJ네트웍스는 300억원 규모의 녹색채권을 발행했다고 28일 밝...
5,AJ네트웍스,2024-05-24 08:50:00,"교보증권 ""AJ네트웍스, 로봇 시장 활성화 역할 기대""",[\n\n\n\n\nAJ네트웍스 로고[회사 웹사이트 캡처] (서울=연합뉴스)...
6,AJ네트웍스,2024-04-04 10:06:32,"AJ네트웍스, 로봇 렌탈 사업으로 한 단계 도약한다",[\n\t\t\t◆ 2024 대한민국 최고의 경영대상 ◆\n\n\n\n AJ네트웍스...
7,AK홀딩스,2024-03-21 17:27:06,"'애경家 장남' 채형석 AK홀딩스 부회장, 작년 연봉 17억 받아",[\n\n\n\n\n애경그룹 채형석 총괄부회장[서울=뉴시스] 이혜원 기자 = 장영신...
8,AK홀딩스,2024-02-15 14:50:01,"AK홀딩스, 지난해 영업익 2850억…흑자전환 성공",[\n\n\n\n\nAK홀딩스가 자회사 실적개선에 힘입어 흑자로 전환했다.AK홀딩스...
9,AK홀딩스,2024-02-15 14:57:04,"AK홀딩스, 주당 200원 현금배당 결정",[\n\t\t\t[헤럴드경제=증권부] AK홀딩스는 보통주 1주당 200원의 현금 결...


In [ ]:
news['list'] = news.apply(lambda row: [row['title'], row['content']], axis=1)
# 기업별로 하나의 행으로 합치기
news2 = news.groupby('corp_name').agg({
    'date': 'first',  # 최신 날짜를 유지
    'list': lambda x: x.tolist()  # 'list' 열의 모든 항목을 리스트로 결합
}).reset_index()

news2

,corp_name,date,list
0,AJ네트웍스,2024-07-24 16:54:10,"[[[하이일드 조달 노트] '차입구조 개선' AJ네트웍스, '로봇사업 강화' 기반 ..."
1,AK홀딩스,2024-03-21 17:27:06,"[['애경家 장남' 채형석 AK홀딩스 부회장, 작년 연봉 17억 받아, [\n\n\..."
2,BGF,2024-08-07 08:50:54,"[[""3070명 한강 달린다"" CU '모두의 해방, 광복런' 캠페인 진행, [\n\..."
3,BGF리테일,2024-08-07 09:58:08,"[[광복절에 달려볼까…BGF리테일 ‘모두의 해방, 광복RUN’, [\n12일까지 참..."
4,BNK금융지주,2024-07-31 18:16:14,"[[BNK금융그룹, 예상 넘어선 2분기 순이익 ‘2428억 원’, [\n지난해 동기..."
...,...,...,...
997,효성중공업,2024-08-07 15:57:18,"[[효성, 미국 초고압 변압기 생산기지 증설, [\n\n\n\n\n효성중공업의 미국..."
998,효성첨단소재,2024-07-31 10:19:14,"[[효성첨단소재, 멸종위기에 처한 꿀벌 보호 나선다, [\n울산 울주군∙국립생태원과..."
999,효성티앤씨,2024-08-07 15:57:18,"[[효성, 미국 초고압 변압기 생산기지 증설, [\n\n\n\n\n효성중공업의 미국..."
1000,효성화학,2024-08-09 17:34:18,"[[효성화학, 특수가스사업부 매각 임박...재무위기 숨통, [\n\t\t\t&lt;..."


In [ ]:
# 중첩 리스트를 벗기는 함수
def flatten_list(nested_list):
    flat_list = []
    for item in nested_list:
        if isinstance(item, list):
            flat_list.extend(flatten_list(item))
        else:
            # 항목을 문자열로 변환
            flat_list.append(str(item) if item is not None else '')
    return flat_list

# 모든 행에 대해 중첩 리스트 평탄화 및 쉼표로 연결
news2['flattened'] = news2['list'].apply(lambda x: ','.join(flatten_list(x)))

# 결과 출력
print(news2[['flattened']])

                                              flattened
0     [하이일드 조달 노트] '차입구조 개선' AJ네트웍스, '로봇사업 강화' 기반 마련...
1     '애경家 장남' 채형석 AK홀딩스 부회장, 작년 연봉 17억 받아,[\n\n\n\n...
2     "3070명 한강 달린다" CU '모두의 해방, 광복런' 캠페인 진행,[\n\n\n...
3     광복절에 달려볼까…BGF리테일 ‘모두의 해방, 광복RUN’,[\n12일까지 참가신청...
4     BNK금융그룹, 예상 넘어선 2분기 순이익 ‘2428억 원’,[\n지난해 동기보다 ...
...                                                 ...
997   효성, 미국 초고압 변압기 생산기지 증설,[\n\n\n\n\n효성중공업의 미국 초고...
998   효성첨단소재, 멸종위기에 처한 꿀벌 보호 나선다,[\n울산 울주군∙국립생태원과 멸종...
999   효성, 미국 초고압 변압기 생산기지 증설,[\n\n\n\n\n효성중공업의 미국 초고...
1000  효성화학, 특수가스사업부 매각 임박...재무위기 숨통,[\n\t\t\t&lt;앵커&...
1001  "술고래는 노화도 빠르다?"...술 많이 마실수록 더 늙어, 왜?,[\n알코올 치료...

[1002 rows x 1 columns]


In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize

from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def summarize_text(text):

    # PlaintextParser 및 LsaSummarizer 사용
    parser = PlaintextParser.from_string(news2['flattened'], Tokenizer("english"))
    summarizer = LsaSummarizer()
    summary = summarizer(parser.document, 1)  # 요약문 1문장 생성

    # 요약 결과를 문자열로 변환
    summary_text = ' '.join(str(sentence) for sentence in summary)
    return summary_text

news2['summary']=''

for index, row in news2.iterrows():
    # flattened 열의 텍스트를 요약
    summary_text = summarize_text(row['flattened'])

    # summary 열에 요약 결과 추가
    news2.at[index, 'summary'] = summary_text


In [ ]:
news2

,corp_name,date,list,flattened,summary
0,AJ네트웍스,2024-07-24 16:54:10,"[[[하이일드 조달 노트] '차입구조 개선' AJ네트웍스, '로봇사업 강화' 기반 ...","[하이일드 조달 노트] '차입구조 개선' AJ네트웍스, '로봇사업 강화' 기반 마련...","""...술 많이 마실수록 더 늙어, 왜?,[\n알코올 치료... Name: flat..."
1,AK홀딩스,2024-03-21 17:27:06,"[['애경家 장남' 채형석 AK홀딩스 부회장, 작년 연봉 17억 받아, [\n\n\...","'애경家 장남' 채형석 AK홀딩스 부회장, 작년 연봉 17억 받아,[\n\n\n\n...","""...술 많이 마실수록 더 늙어, 왜?,[\n알코올 치료... Name: flat..."
2,BGF,2024-08-07 08:50:54,"[[""3070명 한강 달린다"" CU '모두의 해방, 광복런' 캠페인 진행, [\n\...","""3070명 한강 달린다"" CU '모두의 해방, 광복런' 캠페인 진행,[\n\n\n...","""...술 많이 마실수록 더 늙어, 왜?,[\n알코올 치료... Name: flat..."
3,BGF리테일,2024-08-07 09:58:08,"[[광복절에 달려볼까…BGF리테일 ‘모두의 해방, 광복RUN’, [\n12일까지 참...","광복절에 달려볼까…BGF리테일 ‘모두의 해방, 광복RUN’,[\n12일까지 참가신청...","""...술 많이 마실수록 더 늙어, 왜?,[\n알코올 치료... Name: flat..."
4,BNK금융지주,2024-07-31 18:16:14,"[[BNK금융그룹, 예상 넘어선 2분기 순이익 ‘2428억 원’, [\n지난해 동기...","BNK금융그룹, 예상 넘어선 2분기 순이익 ‘2428억 원’,[\n지난해 동기보다 ...","""...술 많이 마실수록 더 늙어, 왜?,[\n알코올 치료... Name: flat..."
...,...,...,...,...,...
997,효성중공업,2024-08-07 15:57:18,"[[효성, 미국 초고압 변압기 생산기지 증설, [\n\n\n\n\n효성중공업의 미국...","효성, 미국 초고압 변압기 생산기지 증설,[\n\n\n\n\n효성중공업의 미국 초고...","""...술 많이 마실수록 더 늙어, 왜?,[\n알코올 치료... Name: flat..."
998,효성첨단소재,2024-07-31 10:19:14,"[[효성첨단소재, 멸종위기에 처한 꿀벌 보호 나선다, [\n울산 울주군∙국립생태원과...","효성첨단소재, 멸종위기에 처한 꿀벌 보호 나선다,[\n울산 울주군∙국립생태원과 멸종...","""...술 많이 마실수록 더 늙어, 왜?,[\n알코올 치료... Name: flat..."
999,효성티앤씨,2024-08-07 15:57:18,"[[효성, 미국 초고압 변압기 생산기지 증설, [\n\n\n\n\n효성중공업의 미국...","효성, 미국 초고압 변압기 생산기지 증설,[\n\n\n\n\n효성중공업의 미국 초고...","""...술 많이 마실수록 더 늙어, 왜?,[\n알코올 치료... Name: flat..."
1000,효성화학,2024-08-09 17:34:18,"[[효성화학, 특수가스사업부 매각 임박...재무위기 숨통, [\n\t\t\t&lt;...","효성화학, 특수가스사업부 매각 임박...재무위기 숨통,[\n\t\t\t&lt;앵커&...","""...술 많이 마실수록 더 늙어, 왜?,[\n알코올 치료... Name: flat..."


In [ ]:
for i in range(1002):
  text = news2['flattened'][i]
  parser = PlaintextParser.from_string(text, Tokenizer("english"))
  summarizer = LsaSummarizer()
  summary = summarizer(parser.document, 1)  # 요약문 1문장 생성
  for sentence in summary:
    news2['summary'][i]=sentence

In [ ]:
news2=news2[['corp_name','date','summary']]
news2

,corp_name,date,summary
0,AJ네트웍스,2024-07-24 16:54:10,이후 공모주 하이일드펀드 수요를 겨냥한 사모사채 발행을 통해 지속해서 금리를 낮추는...
1,AK홀딩스,2024-03-21 17:27:06,"발행시장실과 국제금융실 팀장을 거쳐 런던지점장, 벤쳐금융실장을 역임했으며, 창조금융..."
2,BGF,2024-08-07 08:50:54,"BGF리테일은 편의점 부문 실적을 따로 집계하지는 않지만, 매출의 95% 이상이 편..."
3,BGF리테일,2024-08-07 09:58:08,"BGF리테일은 편의점 부문 실적을 따로 집계하지는 않지만, 매출의 95% 이상이 편..."
4,BNK금융지주,2024-07-31 18:16:14,상반기 보통주자본비율은 적정 이익 실현과 경기 하방리스크에 대비한 적극적인 위험가중...
...,...,...,...
997,효성중공업,2024-08-07 15:57:18,통상 변압기 수명이 30년가량임을 고려하면 사용연한 임박에 따라 대규모 교체 수요가...
998,효성첨단소재,2024-07-31 10:19:14,현금자동입출금기(ATM) 제조업체 효성티앤에스가 유럽과 인도네시아 판매 확대에 힘입...
999,효성티앤씨,2024-08-07 15:57:18,현금자동입출금기(ATM) 제조업체 효성티앤에스가 유럽과 인도네시아 판매 확대에 힘입...
1000,효성화학,2024-08-09 17:34:18,"조현준 회장은 기존 지주사인 ㈜효성과 함께 효성중공업, 효성티앤씨, 효성화학 등을 ..."


In [ ]:
news2.to_excel('df_news7.xlsx',index=False)